In [1]:
import numpy as np
import pandas as pd
import mlflow
import dagshub
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report,confusion_matrix
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import logging
import os
import time


In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [4]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
0,every great gangster movie under current human...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino s partner crime roger avary ...,negative
4,sat tv hoping name would worth time but dear g...,negative


In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,every great gangster movie under current human...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino s partner crime roger avary ...,0
4,sat tv hoping name would worth time but dear g...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
mlflow.set_tracking_uri("https://dagshub.com/Jugal-lachhwani/Data-Capstone-Project.mlflow")
dagshub.init(repo_owner='Jugal-lachhwani', repo_name='Data-Capstone-Project', mlflow=True)

Accessing as Jugal-lachhwani

Initialized MLflow to track repo "Jugal-lachhwani/Data-Capstone-Project"

Repository Jugal-lachhwani/Data-Capstone-Project initialized!

In [12]:
mlflow.set_experiment("Logistic Regression Baseline")

<Experiment: artifact_location='mlflow-artifacts:/6e52a587d59c4f7987160b71fe653f07', creation_time=1760846762471, experiment_id='0', last_update_time=1760846762471, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
# !pip install mlflow==2.16.2

In [14]:
# Check MLflow version and disable model registry features for DagHub compatibility
import mlflow
print(f"MLflow version: {mlflow.__version__}")

# Disable the logged model feature that DagHub doesn't support
os.environ["MLFLOW_ENABLE_LOGGED_MODEL_CREATION"] = "false"

MLflow version: 2.16.2


In [20]:
import sys

# Reset logging configuration for notebook
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout)
    ],
    force=True  # Force reconfiguration
)

params = {'random_state':42,'max_iter':1000}

logging.info("trying to connect to mlflow...")

with mlflow.start_run():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    start_time = time.time()
    try:
        logging.info("Logging preprocessing parameters in mlflow...")
        mlflow.log_param("vectorizer","Bag of words")
        mlflow.log_param("num_features",100)
        mlflow.log_param("test_size",0.25)
        
        logging.info("Intializing logistic regression model...")
        model = LogisticRegression(max_iter=1000,random_state=42)
        mlflow.log_param("model","LogisticRegression")
        mlflow.log_param("max_iter_logistic",1000)
        mlflow.log_param("random_state",42)
        mlflow.log_dict(params,'params.json')
        
        logging.info("Training...")
        model.fit(X_train,y_train)
        
        logging.info("Make predictions...")
        y_pred = model.predict(X_test)
        
        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")
        logging.info(f"Classification Report:\n{classification_report(y_test, y_pred)}")
        
        logging.info("preparing confusion_metrics")
        logging.info(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
        
        logging.info("Saving and logging the model...")
        # Save model without model registry (DagHub compatible)
        mlflow.sklearn.log_model(model, "model")

        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-19 12:26:20,671 - INFO - trying to connect to mlflow...
2025-10-19 12:26:22,001 - INFO - Logging preprocessing parameters in mlflow...
2025-10-19 12:26:22,001 - INFO - Logging preprocessing parameters in mlflow...
2025-10-19 12:26:23,172 - INFO - Intializing logistic regression model...
2025-10-19 12:26:23,172 - INFO - Intializing logistic regression model...
2025-10-19 12:26:25,269 - INFO - Training...
2025-10-19 12:26:25,269 - INFO - Training...
2025-10-19 12:26:25,326 - INFO - Make predictions...
2025-10-19 12:26:25,328 - INFO - Calculating evaluation metrics...
2025-10-19 12:26:25,326 - INFO - Make predictions...
2025-10-19 12:26:25,328 - INFO - Calculating evaluation metrics...
2025-10-19 12:26:25,349 - INFO - Logging evaluation metrics...
2025-10-19 12:26:25,349 - INFO - Logging evaluation metrics...
2025-10-19 12:26:26,942 - INFO - Accuracy: 0.64
2025-10-19 12:26:26,942 - INFO - Precision: 0.6610169491525424
2025-10-19 12:26:26,944 - INFO - Recall: 0.609375
2025-10-19 12

2025/10/19 12:26:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


2025-10-19 12:26:37,750 - INFO - Model training and logging completed in 15.75 seconds.


2025/10/19 12:26:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run mercurial-hen-120 at: https://dagshub.com/Jugal-lachhwani/Data-Capstone-Project.mlflow/#/experiments/0/runs/d064ac937ebd4aee8c2b5fddbfaed9d2.
2025/10/19 12:26:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Jugal-lachhwani/Data-Capstone-Project.mlflow/#/experiments/0.
2025/10/19 12:26:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Jugal-lachhwani/Data-Capstone-Project.mlflow/#/experiments/0.


In [23]:
from datetime import datetime
datetime.now().strftime('%m-%d-%Y-%H-%M-%S')

'10-19-2025-18-28-15'